In [6]:
!pip install pytesseract

In [7]:
!pip install opencv-python

In [8]:
!pip install pymupdf

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import fitz
import pytesseract
from PIL import Image

In [10]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [11]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

In [12]:
# Test image upload
image_path = 'image.png'
image = Image.open(image_path)

# Russian language support
custom_config = r'--oem 1 --psm 1 -l rus'  # rus - код языка русского

# Direct version of extraction
table_text = pytesseract.image_to_string(image, config=custom_config)
print(table_text)

Облучаемая Коэффициент риска Коэффициент риска Сумма,
группа злокачественных наследственных х10-2 Зв-1
населения новообразований, эффектов,
х10-2 Зв-1 х10-2 Зв-1
Все население 5,5 0,2 5,7
Взрослые 4,1 0,1 4,2




In [13]:
# Split to rows
rows = table_text.split('\n')

# Split to elements
table_data = [row.split('\t') for row in rows]

# Make a dataframe
df = pd.DataFrame(table_data)
print(df)

                                                   0
0  Облучаемая Коэффициент риска Коэффициент риска...
1   группа злокачественных наследственных х10-2 Зв-1
2               населения новообразований, эффектов,
3                              х10-2 Зв-1 х10-2 Зв-1
4                          Все население 5,5 0,2 5,7
5                               Взрослые 4,1 0,1 4,2
6                                                   
7                                                   


In [99]:
# In case of .pdf file
pages = []
file = 'test.pdf'
doc = fitz.open(file)
for n in range(doc.page_count-1):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    pages.append(image)

In [100]:
# Remove color info
gray_image= image[:,:,0]
print(type(gray_image))

<class 'numpy.ndarray'>


In [78]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image,180,255,cv2.THRESH_BINARY_INV)

In [79]:
# Dilating image some kind of binary blur
kernel = np.ones((3,3),np.uint8)
dilated_value = cv2.dilate(thresh_value,kernel, iterations = 1)

In [80]:
# Looking for contours, get list of contours and its hierarchy structure
contours, hierarchy = cv2.findContours(dilated_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [81]:
# Extracting coordinates and filtering them empirically
coordinates = [] # our target list of contours
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour) # return bounding box of contour
    if (h>20 or w>20) and (w/h > 2 or h/w > 2):
        coordinates.append((x,y,w,h))

coordinates

[(87, 82, 513, 176),
 (89, 225, 283, 31),
 (288, 232, 39, 10),
 (226, 232, 60, 10),
 (204, 232, 21, 8),
 (133, 232, 70, 10),
 (94, 229, 37, 13),
 (518, 209, 57, 15),
 (470, 209, 46, 15),
 (421, 209, 47, 15),
 (373, 209, 47, 15),
 (89, 209, 283, 15),
 (94, 212, 86, 10),
 (518, 193, 57, 15),
 (470, 193, 46, 15),
 (421, 193, 47, 15),
 (373, 193, 47, 15),
 (89, 193, 283, 15),
 (518, 176, 57, 16),
 (470, 176, 46, 16),
 (421, 176, 47, 16),
 (373, 176, 47, 16),
 (89, 176, 283, 16),
 (94, 180, 57, 10),
 (518, 160, 57, 15),
 (470, 160, 46, 15),
 (421, 160, 47, 15),
 (373, 160, 47, 15),
 (89, 160, 283, 15),
 (518, 144, 57, 15),
 (470, 144, 46, 15),
 (421, 144, 47, 15),
 (373, 144, 47, 15),
 (89, 144, 283, 15),
 (518, 128, 57, 15),
 (470, 128, 46, 15),
 (421, 128, 47, 15),
 (373, 128, 47, 15),
 (89, 128, 283, 15),
 (110, 120, 33, 7),
 (518, 111, 57, 16),
 (535, 114, 25, 11),
 (470, 111, 46, 16),
 (481, 114, 25, 11),
 (421, 111, 47, 16),
 (433, 114, 26, 11),
 (373, 111, 47, 16),
 (385, 114, 26, 11

Попытка удалить символы с картинки

In [101]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image,180,255,cv2.THRESH_BINARY_INV)

In [108]:
# Looking for contours, get list of contours and its hierarchy structure
contours, hierarchy = cv2.findContours(dilated_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# Create a mask image for the characters
mask = np.ones_like(gray_image, dtype=np.uint8) * 255

# Extracting coordinates and filtering them empirically
coordinates = [] # our target list of contours
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour) # return bounding box of contour
    if h<100 and w<100:
        coordinates.append((x,y,w,h))

coordinates

[(594, 218, 5, 3),
 (594, 186, 5, 3),
 (590, 132, 10, 9),
 (588, 114, 12, 11),
 (564, 253, 3, 3),
 (505, 252, 5, 4),
 (457, 252, 5, 4),
 (567, 248, 3, 3),
 (561, 248, 3, 3),
 (138, 248, 3, 3),
 (131, 248, 6, 3),
 (518, 225, 57, 31),
 (470, 225, 46, 31),
 (421, 225, 47, 31),
 (373, 225, 47, 31),
 (288, 232, 39, 10),
 (294, 237, 3, 3),
 (226, 232, 60, 10),
 (263, 234, 6, 3),
 (204, 232, 21, 8),
 (133, 232, 70, 10),
 (158, 235, 3, 3),
 (94, 229, 37, 13),
 (100, 234, 4, 4),
 (564, 221, 3, 3),
 (505, 221, 3, 3),
 (567, 216, 3, 3),
 (561, 216, 3, 3),
 (552, 216, 3, 3),
 (508, 216, 3, 3),
 (502, 216, 3, 3),
 (518, 209, 57, 15),
 (470, 209, 46, 15),
 (421, 209, 47, 15),
 (373, 209, 47, 15),
 (94, 212, 86, 10),
 (165, 217, 3, 3),
 (161, 217, 3, 3),
 (145, 217, 4, 3),
 (134, 217, 3, 3),
 (564, 205, 3, 3),
 (409, 205, 3, 3),
 (281, 205, 3, 3),
 (212, 205, 3, 3),
 (190, 205, 3, 3),
 (172, 205, 3, 3),
 (97, 205, 5, 3),
 (504, 204, 3, 4),
 (258, 204, 4, 4),
 (145, 203, 4, 5),
 (260, 202, 3, 3),
 (16

In [109]:
# Remove color info
gray_image = image[:, :, 0].astype(np.uint8)

# Перебор координат контуров
for (x, y, w, h) in coordinates:
    # Отрисовка прямоугольника вокруг контура
    cv2.rectangle(gray_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Отображение изображения с нарисованными прямоугольниками
cv2.imshow('Image with Lines', gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [116]:
# Загрузка изображения с квитанцией
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Адаптивная бинаризация для выделения текста
binary_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Фильтрация контуров для удаления шумов
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
filtered_contours = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 10 and h > 10:  # Фильтрация маленьких контуров
        filtered_contours.append(contour)

print(filtered_contours)

# Создание маски для удаления текста
mask = np.ones(gray_image.shape, dtype=np.uint8) * 255
cv2.drawContours(mask, filtered_contours, -1, 0, thickness=cv2.FILLED)

# Применение маски к исходному изображению
result = cv2.bitwise_and(image, image, mask=mask)

# Отображение результата
cv2.imshow('Результат', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[  8,   6]],

       [[  8, 176]],

       [[839, 176]],

       [[839,   6]]], dtype=int32)]


In [120]:
# Открытие PDF-файла
file = 'image.pdf'
doc = fitz.open(file)

# Перебор страниц PDF-файла
for page_num in range(len(doc)):
    page = doc[page_num]

    # Получение списка блоков текста на странице
    blocks = page.get_text("blocks")

    # Удаление текста из блоков
    for block in blocks:
        if block[4] == 0:
            # Установка пустого текста для блока
            block[4] = ""

    # Получение списка изображений на странице
    images = page.get_page_image_blocks()

    # Удаление изображений на странице
    for image in images:
        xref = image[0]
        page.delete_image(xref)

    # Сохранение изменений в странице
    page.update_page()

# Сохранение изменений в PDF-файле
output_file = 'квитанция_no_text.pdf'
doc.save(output_file)
doc.close()

AttributeError: 'Page' object has no attribute 'get_page_image_blocks'

In [138]:
# Загрузка изображения
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Распознавание текста с помощью Tesseract
text = pytesseract.image_to_string(gray_image, lang='rus')

# Получение ограничивающих рамок символов
boxes = pytesseract.image_to_boxes(gray_image, lang='rus')

# Вывод распознанного текста
print("Распознанный текст:")
print(text)

height, width, _ = image.shape
# Вывод ограничивающих рамок символов
for box in boxes.splitlines():
    box = box.split(' ')
    x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
    cv2.rectangle(image, (x, y), (w, h), (0, 0, 255), 1)

# Отображение изображения с ограничивающими рамками символов
cv2.imshow('Image with Character Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Распознанный текст:
Облучаемая Коэффициент риска Коэффициент риска Сумма,
группа злокачественных наследственных х10-2 Зв-1
населения новообразований, эффектов,
х10-2 Зв-1 х10-2 Зв-1
Все население 5,5 0,2 5,7
Взрослые 4,1 0,1 4,2




Конец идеи

In [95]:
# Apply the mask to the original image to remove the characters
result = cv2.bitwise_and(image, image, mask=mask)

# Display the result
cv2.imshow('Table without Characters', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [82]:
# Remove color info
gray_image = image[:, :, 0].astype(np.uint8)

# Перебор координат контуров
for (x, y, w, h) in coordinates:
    # Отрисовка прямоугольника вокруг контура
    cv2.rectangle(gray_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Отображение изображения с нарисованными прямоугольниками
cv2.imshow('Image with Lines', gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
# Thresholding image, ret = value, thresh_value = new binary image
ret, thresh_value = cv2.threshold(gray_image, 180, 255, cv2.THRESH_BINARY_INV)

# Dilating image some kind of binary blur
kernel = np.ones((2,2),np.uint8)
dilated_value = cv2.dilate(thresh_value,kernel, iterations = 2)

In [58]:
cv2.imshow('Image with Lines', dilated_value)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [226]:
recognized_table = row = []
prev_y = 0
coordinates.sort() #sort by x
coordinates.sort(key = lambda x:x[1]) # sort by y
for coord in coordinates:
    x,y,w,h = coord
    if y>prev_y+100: #new row if y is changed
        recognized_table.append(row)
        row = []
    crop_img = image[y:y+h, x:x+w]
    custom_config = r'--oem 1 --psm 1 -l rus'  # rus - код языка русского
    recognized_string = pytesseract.image_to_string(crop_img, config=custom_config)
    row.append(recognized_string.replace("\n"," "))
    prev_y = y

recognized_table

['Облучаемая | Коэффициент риска | Коэффициент риска ‘Сумма, группа злокачественных наследственных х10-2 Зв-1 населения новообразований, эффектов, х10-2 38-1 х10-2 Зв-1 Все население. 55 02 57 Взрослые 41 ол 4.2  ',
 "'Облучаемая  группа населения  ",
 'Коэффициент риска злокачественных новообразований,  х10-2 Зв-1  ',
 'Коэффициент риска  наследственных  эффектов, х10-2 Зв-1  ',
 '‘Сумма, х10-2 Зв-1  ',
 'Все население ',
 '',
 '',
 '',
 'Взрослые ',
 '',
 '0,1 ',
 '']